# Serap YALÇIN
NLP Course Project 
29th March 2020 

In [387]:
import numpy as np
import pandas as pd
import re
import json
from pprint import pprint
from snowballstemmer import TurkishStemmer
import nltk
#nltk.download("stopwords")
from nltk.util import ngrams
from nltk.collocations import BigramCollocationFinder
from nltk.metrics import BigramAssocMeasures

# 1- Read the dataset and parse it (i.e. tokenization, stemming, stopwords removal). (10+10= 20% of the assignment)


In [388]:
##READ the Datasets 
data=pd.read_excel(r"C:\Users\budaks\Desktop\YL 2020 Bahar\NLP\MidtermProject\cv_listesi.xlsx")
data.columns = ['No', 'RefN','Resume','Yetenekler','ResumeAll']
##ResumeAll includes all of the information coming from other columns
data = data[['No','RefN','ResumeAll']]
data['ResumeAll']=data['ResumeAll'].str.lower()

data.head()

,No,RefN,ResumeAll
0,0,473,- mapco ve turkcellgroup bünyesindeki şirketle...
1,1,900,<p>aile şirketimizde ve inşaat sektöründe paza...
2,2,1323,"bilgi i̇şlem yöneticisi.&nbsp;moda tekstil, ro..."
3,3,3748,- setting up departments in the production fac...
4,4,6363,<p>i̇ç satın alma faaliyetlerinden sorumlu sat...


In [389]:
len(data)

8012

In [390]:
#Read Ads
adsdata=pd.read_excel(r"C:\Users\budaks\Desktop\YL 2020 Bahar\NLP\MidtermProject\ilan_içerikleri.xlsx")
adsdata.columns = ['AdsNo', 'JobId','Ilan']
adsdata['Ilan']=adsdata['Ilan'].str.lower()
adsdata.head()

,AdsNo,JobId,Ilan
0,0,1870838,makam şoförü \r\n<ul>\r\n<li>en az 5 yıl makam...
1,1,1676048,şoför / sürücü avrupa bölge müdürlüğü'ne bağlı...
2,2,2141993,şoför / sürücü sitil çizgi şirketler grubu bün...
3,3,2175311,şoför / sürücü <strong>alfred concierge hizmet...
4,4,2193892,şoför / sürücü <ul><li><span><span>yoğun iş te...


In [391]:
##Create functions to remove punctuation, tokenize, remove stopwords, and stem
import re 
  
def remove(string): 
    pattern = re.compile(r'[,.;@#?!\&$!"#$%&()*+, -./\':;<=>?@[\]^_`{|}~]+\ *') 
    
    return re.sub(pattern, ' ', string)

In [392]:
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

In [393]:
##Create functions to remove punctuation, tokenize, remove stopwords, and stem
turkish_stops=set(stopwords.words("turkish"))
stopwords = set(stopwords.words('english'))
def clean_text(statement):
    
    tokenizer = RegexpTokenizer(r"\w+")
    statement_tokens = tokenizer.tokenize(str(statement))

   ## ps = nltk.PorterStemmer()
    TurkStem=TurkishStemmer()
    
    statements_clean = []     

    for word in statement_tokens:
        if word not in turkish_stops:
            if word not in stopwords:
                stem_word = TurkStem.stemWord(word)
                statements_clean.append(stem_word)
    return statements_clean

In [394]:
data['ResumeAllClean'] = data['ResumeAll'].apply(lambda x: remove(x))
adsdata['IlanClean'] = adsdata['Ilan'].apply(lambda x: remove(x))

In [395]:
data.head()

,No,RefN,ResumeAll,ResumeAllClean
0,0,473,- mapco ve turkcellgroup bünyesindeki şirketle...,mapco ve turkcellgroup bünyesindeki şirketler...
1,1,900,<p>aile şirketimizde ve inşaat sektöründe paza...,p aile şirketimizde ve inşaat sektöründe paza...
2,2,1323,"bilgi i̇şlem yöneticisi.&nbsp;moda tekstil, ro...",bilgi i̇şlem yöneticisi nbsp moda tekstil rote...
3,3,3748,- setting up departments in the production fac...,setting up departments in the production faci...
4,4,6363,<p>i̇ç satın alma faaliyetlerinden sorumlu sat...,p i̇ç satın alma faaliyetlerinden sorumlu sat...


In [396]:
adsdata.head()

,AdsNo,JobId,Ilan,IlanClean
0,0,1870838,makam şoförü \r\n<ul>\r\n<li>en az 5 yıl makam...,makam şoförü \r\n ul \r\n li en az 5 yıl makam...
1,1,1676048,şoför / sürücü avrupa bölge müdürlüğü'ne bağlı...,şoför sürücü avrupa bölge müdürlüğü ne bağlı ş...
2,2,2141993,şoför / sürücü sitil çizgi şirketler grubu bün...,şoför sürücü sitil çizgi şirketler grubu bünye...
3,3,2175311,şoför / sürücü <strong>alfred concierge hizmet...,şoför sürücü strong alfred concierge hizmetler...
4,4,2193892,şoför / sürücü <ul><li><span><span>yoğun iş te...,şoför sürücü ul li span span yoğun iş temposun...


In [397]:
data["ResumeAllClean"]=data["ResumeAllClean"].map(clean_text)
adsdata["IlanClean"]=adsdata["IlanClean"].map(clean_text)

In [398]:
data.head()

,No,RefN,ResumeAll,ResumeAllClean
0,0,473,- mapco ve turkcellgroup bünyesindeki şirketle...,"[mapco, turkcellgroup, bünyes, şirket, ms, bac..."
1,1,900,<p>aile şirketimizde ve inşaat sektöründe paza...,"[p, ai, şirket, inşaat, sektör, pazarla, satış..."
2,2,1323,"bilgi i̇şlem yöneticisi.&nbsp;moda tekstil, ro...","[bilgi, şlem, yöneticis, nbsp, mo, tekstil, ro..."
3,3,3748,- setting up departments in the production fac...,"[settingi, departments, production, facility, ..."
4,4,6363,<p>i̇ç satın alma faaliyetlerinden sorumlu sat...,"[p, ç, sat, alma, faaliyet, sorumlu, sat, alma..."


In [399]:
adsdata.head()

,AdsNo,JobId,Ilan,IlanClean
0,0,1870838,makam şoförü \r\n<ul>\r\n<li>en az 5 yıl makam...,"[maka, şoför, ul, li, 5, yıl, maka, şoförlük, ..."
1,1,1676048,şoför / sürücü avrupa bölge müdürlüğü'ne bağlı...,"[şoför, sürüç, avrup, bölge, müdürlük, bağlı, ..."
2,2,2141993,şoför / sürücü sitil çizgi şirketler grubu bün...,"[şoför, sürüç, sitil, çizgi, şirket, grup, bün..."
3,3,2175311,şoför / sürücü <strong>alfred concierge hizmet...,"[şoför, sürüç, strong, alfredi, concierge, hiz..."
4,4,2193892,şoför / sürücü <ul><li><span><span>yoğun iş te...,"[şoför, sürüç, ul, li, span, span, yok, iş, te..."


In [400]:
data.ResumeAllClean[1]

['p',
 'ai',
 'şirket',
 'inşaat',
 'sektör',
 'pazarla',
 'satış',
 'destek',
 'birimlerin',
 'satış',
 'strateji',
 'finansma',
 'siste',
 'entegrasyo',
 'şirke',
 'ait',
 'izmir',
 'ankar',
 'stanbul',
 'teknoloj',
 'alt',
 'yapı',
 'oluşturulmas',
 'iş',
 'strateji',
 'hedef',
 'doğrultu',
 'belirlene',
 'finans',
 'nsan',
 'kaynak',
 'sat',
 'alma',
 'bak',
 'kali',
 'müşter',
 'hizmet',
 'ait',
 'kurumsal',
 'iş',
 'uygulamas',
 'ihtiyaç',
 'analiz',
 'edilmes',
 'iş',
 'çözüm',
 'belirlenmes',
 'p',
 'bilgisayar',
 'programcılık',
 'mezuniyet',
 'dona',
 'onar',
 'teknik',
 'destek',
 'eğitim',
 'hilto',
 'universty',
 'management',
 'levels',
 'trainingi',
 'sertifika',
 'şletme',
 'bilgiişle',
 'entegrasyo',
 'konu',
 'uluslararas',
 'ulusal',
 'çap',
 'firma',
 'geniş',
 'yok',
 'süreçli',
 'iş',
 'tecrübes',
 '25',
 'yıllık',
 'teknik',
 'destek',
 'yazıl',
 'tecrübes',
 'p',
 'hilto',
 'standart',
 'kur',
 'dijital',
 'dönüş',
 'süreç',
 'liderlik',
 'edilmes',
 'holdingi',

In [401]:
data['ResumeAllClean2'] = data['ResumeAllClean']
adsdata['IlanClean2'] = adsdata['IlanClean']

In [ ]:

for i in data.ResumeAllClean.index:
    data.ResumeAllClean2[i]=' '.join(data.ResumeAllClean[i])
    
##data.ResumeAllClean2[0]

C:\Users\budaks\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [355]:
for i in adsdata.IlanClean.index:
    adsdata.IlanClean2[i]=' '.join(adsdata.IlanClean[i])
    


C:\Users\budaks\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [376]:
adsdata.IlanClean2[0]

'maka şoför ul li 5 yıl maka şoförlük tecrübe sahip li li diksiyo düzgü iletiş beceris yük li li dış görünüm öze göstere li li  sürüş teknik konu bilgil deneyimli li li b sınıf sürüç belge sahip aktif otomobil kullanabile li li esnek çalışma saat u sağlayabilecek li li askerlik hizmet tamamlamış li li strong anadol yaka strong tercihe suadi nbsp ikamet e li li referans verebilecek li ul br br h3 ş tan h3 ul li bağlı bulunduk yönetiç taraf şirket iç dış verile görev sorumluluk üstlenebilecek li li tahsis edile araç trafik kural uygu olarak kullanım öze gösterecek li li kullanmak olduk araç periyodik bakım zaman eksiksiz bir şekil yer getirilme sağlamak li li kişisel hijye temizlik kural uyarak belirtile standart çerçeve hizmet vermek li ul'

In [377]:
data.ResumeAllClean2[0]

'mapco turkcellgroup bünyes şirket ms backoffiç server w2k exchange isa sql client kurul bak tamir 150 civar pc 30 server çevre birim yazıç scanner vs br pc cevre birim ayak tutulmas kullanıcı destek verilmes br lan wan kontrolü network ha raporlamas performans iyileştirmes br inet mail dış network bağlantı ayak tutulmas superonl turkcell da center buluna microsoft intel microsoft backoffiç small backoffiç ms offiç windows islet sistem server client network kablola server access linux proxy firewall client hp dell 2003 windows işlet 2002 activ directory exchange isa cisco cihaz wireless network adsl microsoft backoffiç small backoffiç ms offiç windows islet sistem server client network kablola server access linux proxy firewall client hp dell 2003 windows işlet 2002 activ directory exchange isa cisco cihaz wireless network adsl p 11 hastane 3500 pc 100 sunuç 200 switch firewall diğer siste network ekiplan 30 bilgi sistem personel sorumluluk p p sahip olduk süreç performans ölçümlemek t

In [356]:
data.ResumeAllClean2.str.split(expand=True).stack().value_counts()


p            857
li           371
br           363
nbsp         339
yönet        302
strong       208
proje        195
windows      171
satış        169
olarak       153
offiç        148
ş            148
ms           147
sistem       146
server       141
microsoft    140
çalışma      135
iş           127
kurul        126
span         118
kali         115
şirket       114
excel        111
word         110
style        110
güvenlik     106
program      105
süreç        100
takip         99
ye            92
            ... 
contex         1
v3             1
menşeil        1
ilav           1
mix            1
ister          1
sara           1
cad            1
mobilya        1
2015           1
veri           1
work           1
cafe           1
yaral          1
nda            1
etiyopya       1
macworldu      1
şematik        1
pcs            1
mercedes       1
10000          1
etkiledik      1
mrp2           1
beanstalk      1
agor           1
lçe            1
ald            1
380           

In [357]:
adsdata.IlanClean2.str.split(expand=True).stack().value_counts()

li              19249
span            10342
nbsp             9476
x                8521
p                6356
br               5420
strong           3920
ul               2714
h3               1680
sahip            1277
satış            1247
çalışma          1237
ş                1181
müşter            981
ola               921
em                881
class             854
tan               848
mez               824
hizmet            774
tercihe           761
u                 742
bir               724
olarak            719
konu              714
yıl               714
font              642
takip             622
msonormal         607
aday              585
                ...  
peraka              1
plenecek            1
screen              1
bakarak             1
büyütüyor           1
mba                 1
alignedi            1
clarify             1
tanışal             1
rewardingi          1
blazer              1
konumla             1
mcitp               1
yıllı               1
çeşme     

In [359]:
# Remove unwanted word to get better running performance 
unwantedwords = ['ol','yap','bir','p','li','br','nbsp','strong','n','al','ş','ye']


word_tokens = data['ResumeAllClean']

for i in data.ResumeAllClean.index:
    word_tokens = data.ResumeAllClean[i]
    removing_stopwords = [word for word in word_tokens if word not in unwantedwords]
    data.ResumeAllClean[i] = removing_stopwords


C:\Users\budaks\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [360]:
data.head()

,No,RefN,ResumeAll,ResumeAllClean,ResumeAllClean2
0,0,473,- mapco ve turkcellgroup bünyesindeki şirketle...,"[mapco, turkcellgroup, bünyes, şirket, ms, bac...",mapco turkcellgroup bünyes şirket ms backoffiç...
1,1,900,<p>aile şirketimizde ve inşaat sektöründe paza...,"[ai, şirket, inşaat, sektör, pazarla, satış, d...",p ai şirket inşaat sektör pazarla satış destek...
2,2,1323,"bilgi i̇şlem yöneticisi.&nbsp;moda tekstil, ro...","[bilgi, şlem, yöneticis, mo, tekstil, roteks, ...",bilgi şlem yöneticis nbsp mo tekstil roteks te...
3,3,3748,- setting up departments in the production fac...,"[settingi, departments, production, facility, ...",settingi departments production facility br or...
4,4,6363,<p>i̇ç satın alma faaliyetlerinden sorumlu sat...,"[ç, sat, alma, faaliyet, sorumlu, sat, alma, m...",p ç sat alma faaliyet sorumlu sat alma müdür o...


In [363]:
# Remove unwanted word to get better running performance 
unwantedwordsforads = ['li','span','p','x','br','nbsp','strong','n','al','ş','p','lu','h3','lu','bir','u']


word_tokensads = adsdata['IlanClean']

for i in adsdata.IlanClean.index:
    word_tokensads = adsdata.IlanClean[i]
    removing_stopwords = [word for word in word_tokensads if word not in unwantedwordsforads]
    adsdata.IlanClean[i] = removing_stopwords



C:\Users\budaks\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [364]:
adsdata.head()

,AdsNo,JobId,Ilan,IlanClean,IlanClean2
0,0,1870838,makam şoförü \r\n<ul>\r\n<li>en az 5 yıl makam...,"[maka, şoför, ul, 5, yıl, maka, şoförlük, tecr...",maka şoför ul li 5 yıl maka şoförlük tecrübe s...
1,1,1676048,şoför / sürücü avrupa bölge müdürlüğü'ne bağlı...,"[şoför, sürüç, avrup, bölge, müdürlük, bağlı, ...",şoför sürüç avrup bölge müdürlük bağlı şube de...
2,2,2141993,şoför / sürücü sitil çizgi şirketler grubu bün...,"[şoför, sürüç, sitil, çizgi, şirket, grup, bün...",şoför sürüç sitil çizgi şirket grup bünye cent...
3,3,2175311,şoför / sürücü <strong>alfred concierge hizmet...,"[şoför, sürüç, alfredi, concierge, hizmet, tür...",şoför sürüç strong alfredi concierge hizmet st...
4,4,2193892,şoför / sürücü <ul><li><span><span>yoğun iş te...,"[şoför, sürüç, ul, yok, iş, tempo, esnek, çalı...",şoför sürüç ul li span span yok iş tempo esnek...


# 2- for a CV (cv_listesi.xlsx), encode it using  count vectorizer (required) (10%), then tfidf vectorizer(required) (15%), then RNNs (2%, bonus) document2vec (bonus, 2%). (25% of the assignment)

In [366]:
#Count Vectorizer
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer()
X_counts = count_vect.fit_transform(data['ResumeAllClean2'])
print(X_counts.shape)
print(count_vect.get_feature_names())

(29, 3435)
['000', '000m2', '000sku', '080', '0px', '10', '100', '1000', '10000', '10g', '11', '110', '118', '11g', '12', '120', '1200', '123', '13', '138', '14', '14000', '14001', '15', '150', '1500', '154', '155', '16', '160', '1600', '16949', '1700m2', '17025', '18', '180', '18000', '18001', '1987', '1997', '1999', '20', '200', '2000', '20000', '2002', '2003', '2005', '2007den', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2019', '21', '210', '22', '220', '231', '246', '25', '252', '270', '27001', '2m', '2mkab', '2x3', '30', '300', '3000', '305', '32', '320', '323', '3270', '33', '340', '3500', '351', '360', '365', '380', '3com', '40', '400', '405', '40tl', '42', '43', '45', '450', '48', '500', '52', '53', '55', '560tl', '5651', '5s', '5why', '60', '6426', '670', '70', '71', '8000', '800xa', '820tl', '8217', '826', '855', '8d', '9000', '9001', '9002', '98', '9i', 'abb', 'abbate', 'abone', 'ac', 'accel', 'acceleration', 'access', 'account', 'acenta

In [367]:
print(X_counts.toarray())

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [4 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [368]:
# Tf-Idf
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='utf-8', ngram_range=(1, 3), stop_words=turkish_stops)
features = tfidf.fit_transform(data.ResumeAllClean2).toarray()
features.shape

(29, 362)

In [369]:
print(features)

[[0.14319977 0.         0.05738684 ... 0.04763626 0.04214367 0.03965406]
 [0.         0.         0.         ... 0.08579171 0.06674953 0.        ]
 [0.         0.         0.16039608 ... 0.08695854 0.05480493 0.        ]
 ...
 [0.         0.         0.04134504 ... 0.04688199 0.         0.        ]
 [0.         0.         0.         ... 0.09445128 0.         0.        ]
 [0.07247453 0.06121314 0.         ... 0.06609266 0.         0.        ]]


# 3- For a job ad (ilan_icerikleri.xlsx), encode it using  count vectorizer (required) (10%), then tfidf vectorizer(required) (15%), then RNNs (2%, bonus) document2vec (bonus, 2%). (25% of the assignment)

In [373]:
#Count Vectorizer
from sklearn.feature_extraction.text import CountVectorizer

count_vect2 = CountVectorizer()
Y_counts = count_vect2.fit_transform(adsdata['IlanClean2'])

In [383]:
print(Y_counts.shape)
print(count_vect2.get_feature_names())

(1000, 11400)
['00', '000', '000000', '000in', '000m2', '01', '02', '020', '03a294cc', '05', '07', '08', '09', '0d0d0d', '0pt', '10', '100', '1000', '1000mbps', '105', '10649', '108', '10850', '11', '110', '112', '115', '116', '119', '11pt', '12', '120', '1200', '12000', '125', '1250', '127', '128', '12px', '13', '130', '14', '140', '14000', '14001', '1480', '14px', '15', '150', '1500', '16', '160', '16949', '17', '170', '17021', '17025', '17065', '18', '180', '1800', '186', '1885', '1892', '19', '1903', '1906', '1950', '1953', '1956', '1958', '1964', '1968', '1969', '1971', '1973', '1974', '1976', '1977', '1980', '1982', '1983', '1984', '1985', '1987', '1987den', '1989', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '1yıl', '20', '200', '2000', '2000007629395px', '2000tl', '2001', '2002', '2003', '2005', '2006', '2007', '2008', '2009', '200tl', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '202', '2020', '2021', '2023', '21',

In [384]:
print(Y_counts.toarray())

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 2 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [4 0 0 ... 0 0 0]]


In [385]:
# Tf-Idf
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='utf-8', ngram_range=(1, 3), stop_words=turkish_stops)
featuresads = tfidf.fit_transform(adsdata.IlanClean2).toarray()
featuresads.shape

(1000, 9899)

In [386]:
print(featuresads)

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.1284868  0.07082038 0.         ... 0.         0.         0.        ]]
